# Step 1: Import libraries and directories

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

# Step 2: Import the OHLC dataset into Panadas Dataframe

In [3]:
# Import the dataset into a Pandas Dataframe
tick_df = pd.read_csv(
    Path("../Resources/META.csv"), 
    index_col='Date', 
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
tick_df.head()

C:\Users\b_key\AppData\Local\Temp\ipykernel_16468\3028085274.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tick_df = pd.read_csv(


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-19,141.210007,144.910004,132.500000,133.240005,133.240005,57404900
2018-12-20,130.699997,135.570007,130.000000,133.399994,133.399994,40297900
2018-12-21,133.389999,134.899994,123.419998,124.949997,124.949997,56901500
2018-12-24,123.099998,129.740005,123.019997,124.059998,124.059998,22066000
2018-12-26,126.000000,134.240005,125.889999,134.179993,134.179993,39723400


In [4]:
# Filter the date index and close columns
model_df = tick_df.loc[:, ["High", "Low", "Close"]]

# Drop all NaN values from the DataFrame
model_df = model_df.dropna()

# Review the DataFrame
display(model_df.head())
display(model_df.tail())

,High,Low,Close
Date,,,
2018-12-19,144.910004,132.500000,133.240005
2018-12-20,135.570007,130.000000,133.399994
2018-12-21,134.899994,123.419998,124.949997
2018-12-24,129.740005,123.019997,124.059998
2018-12-26,134.240005,125.889999,134.179993


,High,Low,Close
Date,,,
2023-12-12,334.470001,324.559998,334.220001
2023-12-13,338.369995,332.640015,334.739990
2023-12-14,334.700012,328.640015,333.170013
2023-12-15,338.660004,331.220001,334.920013
2023-12-18,347.559998,337.019989,344.619995


In [5]:
# def generate_stochastic_signals(df, high_col='High', low_col='Low', close_col='Close', k_period=14, d_period=3, overbought=80, oversold=20):
    # tick_df = generate_stochastic_signals(tick_df)
k_period=14
d_period=3
overbought=80
oversold=20

# Calculate %K and %D
model_df['%K'] = 100 * ((model_df['Close'] - model_df['Low'].rolling(window=k_period).min()) / (model_df['High'].rolling(window=k_period).max() - model_df['Low'].rolling(window=k_period).min()))
model_df['%D'] = model_df['%K'].rolling(window=d_period).mean()

# Generate entry/exit signals
model_df['Long Signal'] = (model_df['%K'] < oversold) & (model_df['%D'] < oversold)
model_df['Short Signal'] = (model_df['%K'] > overbought) & (model_df['%D'] > overbought)

In [6]:
model_df["Signals"] = np.where(model_df['Short Signal'], -1,       
np.where(model_df['Long Signal'], 1,0))

In [7]:
# Review the DataFrame
display(model_df.head())
display(model_df.tail())

,High,Low,Close,%K,%D,Long Signal,Short Signal,Signals
Date,,,,,,,,
2018-12-19,144.910004,132.500000,133.240005,NaN,NaN,False,False,0
2018-12-20,135.570007,130.000000,133.399994,NaN,NaN,False,False,0
2018-12-21,134.899994,123.419998,124.949997,NaN,NaN,False,False,0
2018-12-24,129.740005,123.019997,124.059998,NaN,NaN,False,False,0
2018-12-26,134.240005,125.889999,134.179993,NaN,NaN,False,False,0


,High,Low,Close,%K,%D,Long Signal,Short Signal,Signals
Date,,,,,,,,
2023-12-12,334.470001,324.559998,334.220001,70.266544,58.407350,False,False,0
2023-12-13,338.369995,332.640015,334.739990,74.751774,61.577069,False,False,0
2023-12-14,334.700012,328.640015,333.170013,74.352197,73.123505,False,False,0
2023-12-15,338.660004,331.220001,334.920013,81.021407,76.708459,False,False,0
2023-12-18,347.559998,337.019989,344.619995,91.327423,82.233676,False,True,-1


In [8]:
model_df['Signals'].value_counts()

Signals
 0    863
-1    265
 1    130
Name: count, dtype: int64

In [9]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = model_df[['%K', '%D']].shift().dropna()

# Review the DataFrame
X.head()

,%K,%D
Date,,
2019-01-14,93.017020,95.868090
2019-01-15,94.989352,95.233366
2019-01-16,93.021394,93.675922
2019-01-17,85.804698,91.271815
2019-01-18,89.240549,89.355547


In [10]:
# Create the target set selecting the Signal column and assiging it to y
y = model_df['Signals']

# Review the value counts
y.value_counts()

Signals
 0    863
-1    265
 1    130
Name: count, dtype: int64

In [11]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2019-01-14 00:00:00


In [12]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

2019-04-14 00:00:00


In [13]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()

,%K,%D
Date,,
2019-01-14,93.017020,95.868090
2019-01-15,94.989352,95.233366
2019-01-16,93.021394,93.675922
2019-01-17,85.804698,91.271815
2019-01-18,89.240549,89.355547


In [14]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.head()

,%K,%D
Date,,
2019-04-15,96.748473,93.001863
2019-04-16,95.049469,94.008260
2019-04-17,90.506668,94.101537
2019-04-18,87.696138,91.084092
2019-04-22,81.723589,86.642132


In [15]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

iris = datasets.load_iris()

parameters = {
    'kernel':('linear','rbf','sigmoid'),
    'C':[1,5,15,20],
    'degree':[1,5,10,20]
}

svc = svm.SVC()

clf = GridSearchCV(svc, parameters)

clf.fit(iris.data, iris.target)

clf.cv_results_

results = pd.DataFrame(clf.cv_results_)

results.sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
22,0.001327,0.002655,0.000000,0.000000,5,20,rbf,"{'C': 5, 'degree': 20, 'kernel': 'rbf'}",0.966667,1.000000,1.000000,0.966667,1.000000,0.986667,0.016330,1
19,0.001725,0.002995,0.000000,0.000000,5,10,rbf,"{'C': 5, 'degree': 10, 'kernel': 'rbf'}",0.966667,1.000000,1.000000,0.966667,1.000000,0.986667,0.016330,1
16,0.000000,0.000000,0.000000,0.000000,5,5,rbf,"{'C': 5, 'degree': 5, 'kernel': 'rbf'}",0.966667,1.000000,1.000000,0.966667,1.000000,0.986667,0.016330,1
13,0.001790,0.003580,0.000637,0.001273,5,1,rbf,"{'C': 5, 'degree': 1, 'kernel': 'rbf'}",0.966667,1.000000,1.000000,0.966667,1.000000,0.986667,0.016330,1
0,0.000000,0.000000,0.001606,0.003213,1,1,linear,"{'C': 1, 'degree': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.000000,0.980000,0.016330,5
28,0.001600,0.003200,0.000000,0.000000,15,5,rbf,"{'C': 15, 'degree': 5, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.000000,0.980000,0.016330,5
34,0.000000,0.000000,0.001228,0.002457,15,20,rbf,"{'C': 15, 'degree': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.000000,0.980000,0.016330,5
25,0.001600,0.003200,0.001600,0.003201,15,1,rbf,"{'C': 15, 'degree': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.000000,0.980000,0.016330,5
46,0.001590,0.003181,0.000000,0.000000,20,20,rbf,"{'C': 20, 'degree': 20, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.000000,0.980000,0.016330,5
21,0.001766,0.003532,0.000000,0.000000,5,20,linear,"{'C': 5, 'degree': 20, 'kernel': 'linear'}",1.000000,1.000000,0.933333,0.966667,1.000000,0.980000,0.026667,5
